In [6]:
import numpy as np
import pandas as pd

from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
X = iris.data.features 
y = iris.data.targets 
  
# metadata 
# print(iris.metadata)

# variable information
# print(iris.variables)

In [7]:
print(X.shape)

(150, 4)


In [8]:
class KDTree(object):
    
    def __init__(self, points):

        self._dim = points.shape[1]

        self._dist_sq_func = lambda a, b: sum((x - b[i]) ** 2 for i, x in enumerate(a))
                
        def make(points, i=0):
            if len(points) > 1:
                #points.sort(key=lambda x: x[i])
                #print([point[1][i] for point in points])
                #points = points[np.argsort(np.array([point[1][i] for point in points]))]
                points = sorted(points, key=lambda x: x[1][i])
                i = (i + 1) % self._dim
                m = len(points) >> 1
                return [make(points[:m], i), make(points[m + 1:], i), 
                    points[m]]
            if len(points) == 1:
                return [None, None, points[0]]
        
        def add_point(node, point, i=0):
            if node is not None:
                dx = node[2][i] - point[1][i]
                for j, c in ((0, dx >= 0), (1, dx < 0)):
                    if c and node[j] is None:
                        node[j] = [None, None, point]
                    elif c:
                        add_point(node[j], point, (i + 1) % self._dim)

        import heapq
        def get_knn(node, point, k, heap, i=0, depth=1):
            if node is not None:
                dist_sq = self._dist_sq_func(point, node[2][1])
                dx = node[2][1][i] - point[i]
                if len(heap) < k:
                    heapq.heappush(heap, (-dist_sq, depth, node[2]))
                elif dist_sq < -heap[0][0]:
                    heapq.heappushpop(heap, (-dist_sq, depth, node[2]))
                i = (i + 1) % self._dim
                for b in (dx < 0, dx >= 0)[:1 + (dx * dx < -heap[0][0])]:
                    get_knn(node[1 if b else 0], point, k, heap, i, depth + 1)
            if depth == 1:
                return [h[2] for h in sorted(heap)][::-1]

        self._add_point = add_point
        self._get_knn = get_knn 
        self._root = make(list(enumerate(points)))
        
    def add_point(self, point):
        if self._root is None:
            self._root = [None, None, point]
        else:
            self._add_point(self._root, point)

    def get_knn(self, point, k):
        return self._get_knn(self._root, point, k, [])

    def get_nearest(self, point):
        l = self._get_knn(self._root, point, 1, [])
        return l[0] if len(l) else None

In [20]:
from sklearn.model_selection import KFold, train_test_split

X = np.array(iris.data.features)
y = np.array(iris.data.targets)

n_splits = 10

k = int(input("Enter how many neighbours to be considered: "))

X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X, y, test_size=0.2, shuffle=True)

kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

accuracy = np.empty(0)

def getAccuracy(X_train, X_test, y_train, y_test):
    guessed = 0

    kdtree = KDTree(X_train)

    for index, sample in enumerate(X_test):
        pairs = kdtree.get_knn(sample, k)
        indices = [pair[0] for pair in pairs]
        closestK = [X_train[index] for index in indices]
        closestKLabels = y_train[indices]
        types = np.unique(y_train[indices])
        #closestK = X_train[indices[:k]]
        #closestKLabels = y_train[indices[:k]]

        def distToSampleScore(x):
            return 1/(np.linalg.norm(sample - x)+0.001);

        vectorizedDist = np.vectorize(distToSampleScore)

        scores = np.apply_along_axis(distToSampleScore, axis=1, arr=closestK)

        sum_scores = np.array([np.sum(scores[np.array(np.where(closestKLabels == label))[0]]) for label in types])

        res = types[np.argmax(sum_scores)]
    
        #print(f"Test sample: {sample}")
        #print(f"Closest {k} training samples:\n{closest_k}")
        #types, counts = np.unique(closestKLabels, return_counts=True)
        #res_index = np.argmax(counts)
        #res = types[res_index]
        #frequency = counts[res_index]
        #print(f"Res label: {res}")
        guessed += (res == y_test[index])

        #if(res != y_test[index]):
        #    print("wrong")
        #    print(types)
        #    print(sum_scores)
        #    print(y_test[index])
        #    print(res)
        #    print("end wrong")
        #print("-" * 40)

    currentSampleAccuracy = guessed / X_test.shape[0]
    return currentSampleAccuracy

accuracy = np.append(accuracy, getAccuracy(X_train_full, X_train_full, y_train_full, y_train_full))

for fold, (train_index, test_index) in enumerate(kf.split(X_train_full), 1):
    X_train, X_test = X_train_full[train_index], X_train_full[test_index]
    y_train, y_test = y_train_full[train_index], y_train_full[test_index]
    #print(f"Fold {fold}")
    #print(f"Train data shape: {X_train.shape}, Test data shape: {X_test.shape}")
    #print("-" * 40)

    # guessed = 0

    # kdtree = KDTree(X_train)

    # for index, sample in enumerate(X_test):
    #     pairs = kdtree.get_knn(sample, k)
    #     indices = [pair[0] for pair in pairs]
    #     closestK = [X_train[index] for index in indices]
    #     closestKLabels = y_train[indices]
    #     types = np.unique(y_train[indices])
    #     #closestK = X_train[indices[:k]]
    #     #closestKLabels = y_train[indices[:k]]

    #     def distToSampleScore(x):
    #         return 1/(np.linalg.norm(sample - x)+0.001);

    #     vectorizedDist = np.vectorize(distToSampleScore)

    #     scores = np.apply_along_axis(distToSampleScore, axis=1, arr=closestK)

    #     sum_scores = np.array([np.sum(scores[np.array(np.where(closestKLabels == label))[0]]) for label in types])

    #     res = types[np.argmax(sum_scores)]
    
    #     #print(f"Test sample: {sample}")
    #     #print(f"Closest {k} training samples:\n{closest_k}")
    #     #types, counts = np.unique(closestKLabels, return_counts=True)
    #     #res_index = np.argmax(counts)
    #     #res = types[res_index]
    #     #frequency = counts[res_index]
    #     #print(f"Res label: {res}")
    #     guessed += (res == y_test[index])

    #     #if(res != y_test[index]):
    #     #    print("wrong")
    #     #    print(types)
    #     #    print(sum_scores)
    #     #    print(y_test[index])
    #     #    print(res)
    #     #    print("end wrong")
    #     #print("-" * 40)

    # currentSampleAccuracy = guessed / X_test.shape[0]
    # #print(f"Guessed: {currentSampleAccuracy}")
    # accuracy = np.append(accuracy, currentSampleAccuracy)
    accuracy = np.append(accuracy, getAccuracy(X_train, X_test, y_train, y_test))

    #print(accuracy)

Enter how many neighbours to be considered:  4


In [21]:
print(accuracy)
print(np.mean(accuracy))

[1.         0.91666667 1.         1.         1.         1.
 1.         0.91666667 0.83333333 1.         0.83333333]
0.9545454545454546


In [22]:
print(getAccuracy(X_train_full, X_test_full, y_train_full, y_test_full))

[1.]
